In [ ]:
import os
import pandas as pd
import numpy as np
import shutil
import cv2

def np_CountUpContinuingOnes(b_arr):
    left = np.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = np.maximum.accumulate(left)

    rev_arr = b_arr[::-1]
    right = np.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = np.maximum.accumulate(right)
    right = len(rev_arr) - 1 - right[::-1]

    return right - left - 1

def ExtractBreast(img):
    img_copy = img.copy()
    img = np.where(img <= 40, 0, img)
    height, _ = img.shape
    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].std(axis=0) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    col_ind = np.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]
    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].std(axis=1) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    row_ind = np.where(continuing_ones == continuing_ones.max())[0]

    return img_copy[row_ind][:, col_ind]


excel_path = '/Volumes/图图/CDD-CESM/Radiology-manual-annotations.xlsx'
image_dir = '/Volumes/图图/CDD-CESM/PKG - CDD-CESM/CDD-CESM/Low energy images of CDD-CESM'
output_base_dir = 'Benchmark/CDD-CESM'


df = pd.read_excel(excel_path, sheet_name='all')


df = df[df['Image_name'].str.contains('DM')]


def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = ExtractBreast(img)
    img_normalized = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    return img_normalized

def process_all_data(df):

    df = df[['Image_name']]


    os.makedirs(output_base_dir, exist_ok=True)


    for _, row in df.iterrows():
        image_name = row['Image_name'].strip()
        image_path = os.path.join(image_dir, image_name + '.jpg')
        if os.path.exists(image_path):
            img = preprocess_image(image_path)
            

            output_data_dir = os.path.join(output_base_dir, image_name)
            os.makedirs(output_data_dir, exist_ok=True)
            

            cv2.imwrite(os.path.join(output_data_dir, 'img.jpg'), img)

        else:
            print(f"Image not found: {image_path}")


process_all_data(df)

/var/folders/ft/tdx4j_ks1s33pn4xyp1nq6km0000gn/T/ipykernel_99368/3440276096.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class_combination'] = df[['ACR', 'BIRADS', 'Pathology']].apply(lambda x: tuple(x), axis=1)
